In [ ]:
import glob
import os
import shutil

from astropy.io import fits
from astropy import stats
from astropy.time import Time
from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Range1d
from bokeh.layouts import row, column, gridplot
from bokeh.models.widgets import Tabs, Panel
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit

# Determine where the visualization will be rendered
output_notebook() 

In [ ]:
postflash_data = pd.read_pickle('../2022_data/Feb_2022_flc_all_stats_postflash.pkl')

In [ ]:
postflash_data[0:10]

In [ ]:
subarray_pf = postflash_data.loc[(postflash_data['subarray'] == True)] 
fullframe_pf = postflash_data.loc[(postflash_data['subarray'] == False)]
low_sub_pf_A = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'A')] 
low_sub_pf_B = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'B')] 

In [ ]:
fullframe_pf_A = postflash_data.loc[(postflash_data['flash_cur'] == 'MED') & (postflash_data['subarray'] == False) & (postflash_data['shutter'] == 'A') & (postflash_data['flash_lvl'] == 7310)]

In [ ]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(fullframe_pf_A['obsdate'] + ' ' + fullframe_pf_A['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(fullframe_pf_A['norm_by_dur'])
y = fullframe_pf_A['norm_by_dur']/fullframe_pf_A['norm_by_dur'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])
perr = np.sqrt(np.diag(pcov))
print(perr)
print(perr[0]/popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation and Duration',x_axis_type='datetime',plot_width=800, plot_height=300)
p.y_range=Range1d(0.95, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Fullframe, Shutter A")
p.line(x_datetime,y_pred,color='red')#,legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

In [ ]:
fullframe_pf_B = postflash_data.loc[(postflash_data['flash_cur'] == 'MED') & (postflash_data['subarray'] == False) & (postflash_data['shutter'] == 'B') & (postflash_data['flash_lvl'] == 7310)]

In [ ]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(fullframe_pf_B['obsdate'] + ' ' + fullframe_pf_B['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')
print(np.max(x)-np.min(x))

mean_mean = np.mean(fullframe_pf_B['norm_by_dur'])
y = fullframe_pf_B['norm_by_dur']/fullframe_pf_B['norm_by_dur'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])
print(pcov)
perr = np.sqrt(np.diag(pcov))
print(perr)
print(perr[0]/popt[0])


# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation and Duration',x_axis_type='datetime',plot_width=800, plot_height=300)
p.y_range=Range1d(0.95, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Fullframe, Shutter B")
p.line(x_datetime,y_pred,color='red')#,legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)


In [ ]:
low_sub_pf_A_lvl_3 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'A') & (postflash_data['flash_lvl'] == 3)]
low_sub_pf_A_lvl_3

In [ ]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_A_lvl_3['obsdate'] + ' ' + low_sub_pf_A_lvl_3['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_A_lvl_3['sig_mean'])
y = low_sub_pf_A_lvl_3['sig_mean']/low_sub_pf_A_lvl_3['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print(popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean of First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Low Subarray Lvl 3 Shutter A")
p.line(x_datetime,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

In [ ]:
low_sub_pf_B_lvl_3 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'B') & (postflash_data['flash_lvl'] == 3)]

In [ ]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_B_lvl_3['obsdate'] + ' ' + low_sub_pf_B_lvl_3['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_B_lvl_3['sig_mean'])
y = low_sub_pf_B_lvl_3['sig_mean']/low_sub_pf_B_lvl_3['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Low Subarray Lvl 3 Shutter B")
p.line(x_datetime,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

In [ ]:
low_sub_pf_A_lvl_5 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'A') & (postflash_data['flash_lvl'] == 5)]

# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_A_lvl_5['obsdate'] + ' ' + low_sub_pf_A_lvl_5['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_A_lvl_5['sig_mean'])
y = low_sub_pf_A_lvl_5['sig_mean']/low_sub_pf_A_lvl_5['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Low Subarray Lvl 5 Shutter A")
p.line(x_datetime,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

In [ ]:
low_sub_pf_B_lvl_5 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'B') & (postflash_data['flash_lvl'] == 5)]
low_sub_pf_A_lvl_8 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'A') & (postflash_data['flash_lvl'] == 8)]
low_sub_pf_B_lvl_8 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'B') & (postflash_data['flash_lvl'] == 8)]
low_sub_pf_A_lvl_10 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'A') & (postflash_data['flash_lvl'] == 10)]
low_sub_pf_B_lvl_10 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'B') & (postflash_data['flash_lvl'] == 10)]
low_sub_pf_A_lvl_12 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'A') & (postflash_data['flash_lvl'] == 12)]
low_sub_pf_B_lvl_12 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'B') & (postflash_data['flash_lvl'] == 12)]
low_sub_pf_A_lvl_20 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'A') & (postflash_data['flash_lvl'] == 20)]
low_sub_pf_B_lvl_20 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'B') & (postflash_data['flash_lvl'] == 20)]


In [ ]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_B_lvl_5['obsdate'] + ' ' + low_sub_pf_B_lvl_5['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_B_lvl_5['sig_mean'])
y = low_sub_pf_B_lvl_5['sig_mean']/low_sub_pf_B_lvl_5['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Low Subarray Lvl 5 Shutter B")
p.line(x_datetime,y_pred,color='red', legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

In [ ]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_A_lvl_8['obsdate'] + ' ' + low_sub_pf_A_lvl_8['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_A_lvl_8['sig_mean'])
y = low_sub_pf_A_lvl_8['sig_mean']/low_sub_pf_A_lvl_8['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Low Subarray Lvl 8 Shutter A")
p.line(x_datetime,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

In [ ]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_B_lvl_8['obsdate'] + ' ' + low_sub_pf_B_lvl_8['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_B_lvl_8['sig_mean'])
y = low_sub_pf_B_lvl_8['sig_mean']/low_sub_pf_B_lvl_8['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Low Subarray Lvl 8 Shutter B")
p.line(x_datetime,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

In [ ]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_A_lvl_10['obsdate'] + ' ' + low_sub_pf_A_lvl_10['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_A_lvl_10['sig_mean'])
y = low_sub_pf_A_lvl_10['sig_mean']/low_sub_pf_A_lvl_10['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Low Subarray Lvl 10 Shutter A")
p.line(x_datetime,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

In [ ]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_B_lvl_10['obsdate'] + ' ' + low_sub_pf_B_lvl_10['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_B_lvl_10['sig_mean'])
y = low_sub_pf_B_lvl_10['sig_mean']/low_sub_pf_B_lvl_10['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Low Subarray Lvl 10 Shutter B")
p.line(x_datetime,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

In [ ]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_A_lvl_12['obsdate'] + ' ' + low_sub_pf_A_lvl_12['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_A_lvl_12['sig_mean'])
y = low_sub_pf_A_lvl_12['sig_mean']/low_sub_pf_A_lvl_12['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.95, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Low Subarray Lvl 12 Shutter A")
p.line(x_datetime,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

In [ ]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_B_lvl_12['obsdate'] + ' ' + low_sub_pf_B_lvl_12['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_B_lvl_12['sig_mean'])
y = low_sub_pf_B_lvl_12['sig_mean']/low_sub_pf_B_lvl_12['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.1)
p.circle(x_datetime, y,size = 4, legend_label = "Low Subarray Lvl 12 Shutter B")
p.line(x_datetime,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

In [ ]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_A_lvl_20['obsdate'] + ' ' + low_sub_pf_A_lvl_20['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_A_lvl_20['sig_mean'])
y = low_sub_pf_A_lvl_20['sig_mean']/low_sub_pf_A_lvl_20['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.95, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Low Subarray Lvl 20 Shutter A")
p.line(x_datetime,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

In [ ]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_B_lvl_20['obsdate'] + ' ' + low_sub_pf_B_lvl_20['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_B_lvl_20['sig_mean'])
y = low_sub_pf_B_lvl_20['sig_mean']/low_sub_pf_B_lvl_20['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.95, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Subarray, Low Current, Flash Level 20, Shutter B")
p.line(x_datetime,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
low_sub_pf_B_lvl_20.sort_values(by=['obsdate'])


In [ ]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_B_lvl_20['obsdate'] + ' ' + low_sub_pf_B_lvl_20['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_B_lvl_20['norm_by_dur'])
y = low_sub_pf_B_lvl_20['norm_by_dur']/low_sub_pf_B_lvl_20['norm_by_dur'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation and Duration',x_axis_type='datetime',plot_width=800, plot_height=300)
p.y_range=Range1d(0.95, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Subarray, Low Current, Flash Level 20, Shutter B")
p.line(x_datetime,y_pred,color='red')#,legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
p.yaxis.axis_label_text_font_size = '40px'
show(p)

In [ ]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_A_lvl_20['obsdate'] + ' ' + low_sub_pf_A_lvl_20['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_A_lvl_20['norm_by_dur'])
y = low_sub_pf_A_lvl_20['norm_by_dur']/low_sub_pf_A_lvl_20['norm_by_dur'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation and Duration',x_axis_type='datetime',plot_width=800, plot_height=300)
p.y_range=Range1d(0.95, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Subarray, Low Current, Flash Level 20, Shutter A")
p.line(x_datetime,y_pred,color='red')#,legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

In [ ]:
before_low_sub_pf_B_lvl_20 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'B') & (postflash_data['flash_lvl'] == 20) & (postflash_data['flash_dur'] == 7.7)]
after_low_sub_pf_B_lvl_20 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'B') & (postflash_data['flash_lvl'] == 20) & (postflash_data['flash_dur'] == 8.3)]



# normalized to the mean of first observation
b_x_datetime = pd.to_datetime(before_low_sub_pf_B_lvl_20['obsdate'] + ' ' + before_low_sub_pf_B_lvl_20['obstime'])
b_sorted_x = b_x_datetime.sort_values()
first_rootname = b_sorted_x.index[0]
b_x = Time(b_x_datetime,format='datetime64')
b_x.format = 'mjd'
b_x = b_x.to_value('mjd', 'float')

mean_mean = np.mean(before_low_sub_pf_B_lvl_20['sig_mean'])
b_y = before_low_sub_pf_B_lvl_20['sig_mean']/before_low_sub_pf_B_lvl_20['sig_mean'][first_rootname]

# normalized to the mean of first observation
a_x_datetime = pd.to_datetime(after_low_sub_pf_B_lvl_20['obsdate'] + ' ' + after_low_sub_pf_B_lvl_20['obstime'])
a_sorted_x = a_x_datetime.sort_values()
a_first_rootname = a_sorted_x.index[0]
a_x = Time(a_x_datetime,format='datetime64')
a_x.format = 'mjd'
a_x = a_x.to_value('mjd', 'float')

mean_mean = np.mean(after_low_sub_pf_B_lvl_20['sig_mean'])
a_y = after_low_sub_pf_B_lvl_20['sig_mean']/after_low_sub_pf_B_lvl_20['sig_mean'][a_first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, b_x, list(b_y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
b_slope = popt[0] # Takes the first element of the array
b_intercept = popt[1]

# Make the regression line
b_y_pred = []
for i in b_x:
    i = float(i)
    temp_y = slope*i + intercept 
    b_y_pred.append(temp_y)
    
#x = x.reshape(-1, 1)
a_popt, a_pcov = curve_fit(line, a_x, list(a_y))
print("Slope: ", a_popt[0])

# Find the slope and intercept from the model
a_slope = a_popt[0] # Takes the first element of the array
a_intercept = a_popt[1]

# Make the regression line
a_y_pred = []
for i in a_x:
    i = float(i)
    temp_y = slope*i + intercept 
    a_y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.95, 1.05)
p.circle(b_x_datetime, b_y,size = 4, legend_label = "Subarray, Low Current, Flash Level 20, Shutter B")
p.circle(a_x_datetime, a_y,size = 4, legend_label = "Subarray, Low Current, Flash Level 20, Shutter B")
p.line(b_x_datetime,b_y_pred,color='red',legend_label='y = ' + str(b_slope) +' x + '+ str(round(b_intercept,6)))
p.line(a_x_datetime,a_y_pred,color='red',legend_label='y = ' + str(a_slope) +' x + '+ str(round(a_intercept,6)))

show(p)

"The data in Figure 10 are from program 13078 visits A3, A6, A7, and A8 and taken on November 15
and 16, 2012. Images were combined in pairs with STSDAS CRREJ, and the mean pixel value was
computed for the 1024x1024 pixel area nearest the C amplifier using IMSTAT with 3 iterations of 3
sigma rejection. "

In [ ]:
#subarray_mean = pd.read_pickle('all_sub_pf_and_mean_data.pkl')
low_sub_pf_isr_13078_nov15 = subarray_mean.loc[(subarray_mean['flash_cur'] == 'LOW') & (subarray_mean['propid'] == 13078) & (subarray_mean['obsdate'] == '2012-11-15')]
low_sub_pf_isr_13078_nov16 = subarray_mean.loc[(subarray_mean['flash_cur'] == 'LOW') & (subarray_mean['propid'] == 13078) & (subarray_mean['obsdate'] == '2012-11-16')]

both_dates = [low_sub_pf_isr_13078_nov15, low_sub_pf_isr_13078_nov16]

low_sub_pf_isr_13078 = pd.concat(both_dates)
print(low_sub_pf_isr_13078)

In [ ]:
paths = low_sub_pf_isr_13078.path.tolist() 

means = []
sig_means = []
for f in paths:
    path = f
    data = fits.getdata(path)
    data_clip_1 = stats.sigma_clip(data, 3)
    data_clip_2 = stats.sigma_clip(data_clip_1, 3)
    data_clip_3 = stats.sigma_clip(data_clip_2, 3)
    mean = np.mean(data)
    sig_mean = np.mean(data_clip_3)
    means.append(mean)
    sig_means.append(sig_mean)

dict = {'path': paths, 'mean': means, 'sig_mean': sig_means}
path_and_mean_sub = pd.DataFrame(dict)

In [ ]:
low_sub_pf_isr_13078['count_rate'] = (low_sub_pf_isr_13078['sig_mean']/low_sub_pf_isr_13078['flash_dur']) * low_sub_pf_isr_13078['flash_dur']

In [ ]:
low_sub_pf_isr_13078

In [ ]:
med_full_pf = postflash_data.loc[(postflash_data['flash_cur'] == 'MED') & (postflash_data['subarray'] == False)] 

In [ ]:
med_full_pf

In [ ]:
postflash_data = pd.read_pickle('/grp/hst/wfc3u/final_postflash_2021/notebooks/Feb_2022_flc_all_stats_postflash.pkl')

In [ ]:
postflash_data[0:10]

In [ ]:
prop13078 = postflash_data.loc[(postflash_data['propid'] == 13078) & (postflash_data['subarray'] == True)]


In [ ]:
prop13078

In [ ]:
prop13560 = postflash_data.loc[(postflash_data['propid'] == 13560) & (postflash_data['subarray'] == True)]


In [ ]:
prop13560

In [ ]:
prop14006 = postflash_data.loc[(postflash_data['propid'] == 14006) & (postflash_data['subarray'] == True)]

In [ ]:
prop14006